In [1]:
import re
import pandas as pd
import numpy as np
import time
from collections import defaultdict

In [2]:
df_ads = pd.read_csv("YandexAdsad.csv", sep=';', usecols=[1], names=['text'])

twits1 = pd.read_csv('ptwits.csv', sep=';', usecols=[3], names=['text'])
twits2 = pd.read_csv('ntwits.csv', sep=';', usecols=[3], names=['text'])

df_twits = pd.concat([twits1, twits2])

In [3]:
df_twits

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...
...,...
111918,Но не каждый хочет что то исправлять:( http://...
111919,скучаю так :-( только @taaannyaaa вправляет мо...
111920,"Вот и в школу, в говно это идти уже надо("
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :..."


In [4]:
display(df_ads.head())
display(df_twits.head())

,text
0,Доставка по РФ! Доставка транспортной компание...
1,Наличие на складе! Цены и качество от официаль...
2,Доставка по РФ. Оплата при получении! Наличие ...
3,Наличие на складе! Цены и качество от официаль...
4,Buderus! Цены от официального дилера. Гарантия...


,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


In [5]:
df_ads['target'] = 1
df_twits['target'] = 0

ads = df_ads[['text', 'target']]
twits = df_twits[['text', 'target']]

In [6]:
ads

,text,target
0,Доставка по РФ! Доставка транспортной компание...,1
1,Наличие на складе! Цены и качество от официаль...,1
2,Доставка по РФ. Оплата при получении! Наличие ...,1
3,Наличие на складе! Цены и качество от официаль...,1
4,Buderus! Цены от официального дилера. Гарантия...,1
...,...,...
2084087,Для Вашего принтера! Картридж XEROX 108r00909 ...,1
2084088,Для Вашего принтера! Картридж XEROX 3100 MFP к...,1
2084089,Для Вашего принтера! Картридж XEROX 3100MFP Га...,1
2084090,Для Вашего принтера! Картридж XEROX 3117 купит...,1


In [7]:
def preproc(s: str):
    def is_simple(word: str):
        return all((char.isalpha() for char in word))
    return ' '.join({word for word in s.lower().split() if len(word) > 3 and is_simple(word)})

ads['clear'] = ads['text'].apply(preproc)
twits['clear'] = twits['text'].apply(preproc)

In [8]:
df = pd.concat([ads, twits]).sample(frac=1).reset_index(drop=True)
df = df[df['clear'] != '']

In [9]:
len(df)

2291978

In [12]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

TRAIN size: 2062780
TEST size: 229198


In [13]:
!pip install gensim

  Created wheel for smart-open: filename=smart_open-4.1.0-py3-none-any.whl size=106208 sha256=259e841af17a30000f0c267e2e655bf71d76dfa12e00d3c063e56270f9ef67bc
  Stored in directory: c:\users\kroksa\appdata\local\pip\cache\wheels\74\33\8e\37e22f50ce94856f37b3e23a93c648c194aa8d5a546083d09a
Successfully built smart-open
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21


In [14]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [15]:
w2v_model = gensim.models.word2vec.Word2Vec(size=1000, 
                                            window=10, 
                                            min_count=10, 
                                            workers=4)

In [16]:
w2v_model.build_vocab(df['clear'].apply(str.split))

2021-01-17 17:03:39,999 : INFO : collecting all words and their counts
2021-01-17 17:03:40,000 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-17 17:03:40,015 : INFO : PROGRESS: at sentence #10000, processed 41327 words, keeping 10786 word types
2021-01-17 17:03:40,031 : INFO : PROGRESS: at sentence #20000, processed 82629 words, keeping 16426 word types
2021-01-17 17:03:40,043 : INFO : PROGRESS: at sentence #30000, processed 123847 words, keeping 20802 word types
2021-01-17 17:03:40,054 : INFO : PROGRESS: at sentence #40000, processed 165041 words, keeping 24504 word types
2021-01-17 17:03:40,067 : INFO : PROGRESS: at sentence #50000, processed 206419 words, keeping 27822 word types
2021-01-17 17:03:40,082 : INFO : PROGRESS: at sentence #60000, processed 247761 words, keeping 30771 word types
2021-01-17 17:03:40,095 : INFO : PROGRESS: at sentence #70000, processed 289199 words, keeping 33400 word types
2021-01-17 17:03:40,108 : INFO : PROGRESS: at se

2021-01-17 17:03:40,940 : INFO : PROGRESS: at sentence #720000, processed 2975072 words, keeping 111166 word types
2021-01-17 17:03:40,958 : INFO : PROGRESS: at sentence #730000, processed 3016393 words, keeping 111956 word types
2021-01-17 17:03:40,973 : INFO : PROGRESS: at sentence #740000, processed 3057526 words, keeping 112743 word types
2021-01-17 17:03:40,985 : INFO : PROGRESS: at sentence #750000, processed 3098761 words, keeping 113496 word types
2021-01-17 17:03:40,998 : INFO : PROGRESS: at sentence #760000, processed 3139943 words, keeping 114242 word types
2021-01-17 17:03:41,012 : INFO : PROGRESS: at sentence #770000, processed 3181271 words, keeping 115002 word types
2021-01-17 17:03:41,024 : INFO : PROGRESS: at sentence #780000, processed 3222492 words, keeping 115707 word types
2021-01-17 17:03:41,037 : INFO : PROGRESS: at sentence #790000, processed 3263725 words, keeping 116415 word types
2021-01-17 17:03:41,051 : INFO : PROGRESS: at sentence #800000, processed 330505

2021-01-17 17:03:41,929 : INFO : PROGRESS: at sentence #1430000, processed 5904526 words, keeping 155039 word types
2021-01-17 17:03:41,950 : INFO : PROGRESS: at sentence #1440000, processed 5945896 words, keeping 155557 word types
2021-01-17 17:03:41,965 : INFO : PROGRESS: at sentence #1450000, processed 5987459 words, keeping 156076 word types
2021-01-17 17:03:41,979 : INFO : PROGRESS: at sentence #1460000, processed 6029002 words, keeping 156615 word types
2021-01-17 17:03:41,992 : INFO : PROGRESS: at sentence #1470000, processed 6070420 words, keeping 157129 word types
2021-01-17 17:03:42,007 : INFO : PROGRESS: at sentence #1480000, processed 6111785 words, keeping 157625 word types
2021-01-17 17:03:42,019 : INFO : PROGRESS: at sentence #1490000, processed 6153316 words, keeping 158147 word types
2021-01-17 17:03:42,032 : INFO : PROGRESS: at sentence #1500000, processed 6194572 words, keeping 158661 word types
2021-01-17 17:03:42,046 : INFO : PROGRESS: at sentence #1510000, process

2021-01-17 17:03:42,873 : INFO : PROGRESS: at sentence #2140000, processed 8838386 words, keeping 188166 word types
2021-01-17 17:03:42,886 : INFO : PROGRESS: at sentence #2150000, processed 8879774 words, keeping 188624 word types
2021-01-17 17:03:42,899 : INFO : PROGRESS: at sentence #2160000, processed 8921124 words, keeping 189085 word types
2021-01-17 17:03:42,913 : INFO : PROGRESS: at sentence #2170000, processed 8962374 words, keeping 189521 word types
2021-01-17 17:03:42,927 : INFO : PROGRESS: at sentence #2180000, processed 9003600 words, keeping 189929 word types
2021-01-17 17:03:42,940 : INFO : PROGRESS: at sentence #2190000, processed 9044969 words, keeping 190375 word types
2021-01-17 17:03:42,953 : INFO : PROGRESS: at sentence #2200000, processed 9086312 words, keeping 190826 word types
2021-01-17 17:03:42,967 : INFO : PROGRESS: at sentence #2210000, processed 9127530 words, keeping 191192 word types
2021-01-17 17:03:42,982 : INFO : PROGRESS: at sentence #2220000, process

In [17]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 36100


In [18]:
docs = df['clear'].apply(str.split)
w2v_model.train(docs, total_examples=len(docs), epochs=3)

2021-01-17 17:04:43,795 : INFO : training model with 4 workers on 36100 vocabulary and 1000 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2021-01-17 17:04:44,834 : INFO : EPOCH 1 - PROGRESS: at 7.29% examples, 576405 words/s, in_qsize 7, out_qsize 0
2021-01-17 17:04:45,836 : INFO : EPOCH 1 - PROGRESS: at 14.89% examples, 593248 words/s, in_qsize 7, out_qsize 0
2021-01-17 17:04:46,852 : INFO : EPOCH 1 - PROGRESS: at 22.59% examples, 599144 words/s, in_qsize 7, out_qsize 0
2021-01-17 17:04:47,857 : INFO : EPOCH 1 - PROGRESS: at 30.41% examples, 605501 words/s, in_qsize 7, out_qsize 0
2021-01-17 17:04:48,868 : INFO : EPOCH 1 - PROGRESS: at 38.33% examples, 610300 words/s, in_qsize 7, out_qsize 0
2021-01-17 17:04:49,874 : INFO : EPOCH 1 - PROGRESS: at 46.26% examples, 614088 words/s, in_qsize 7, out_qsize 0
2021-01-17 17:04:50,890 : INFO : EPOCH 1 - PROGRESS: at 54.30% examples, 617340 words/s, in_qsize 7, out_qsize 0
2021-01-17 17:04:51,897 : INFO : EPOCH 1 - PROGRESS: at 62

(24124905, 28399998)

In [19]:
w2v_model.most_similar(positive=['хорошо'])

<ipython-input-19-909e824b505e>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_model.most_similar(positive=['хорошо'])
2021-01-17 17:09:28,113 : INFO : precomputing L2-norms of word weight vectors


[('причем', 0.922110915184021),
 ('жаль', 0.9195817708969116),
 ('круто', 0.9181531667709351),
 ('кстати', 0.9100744724273682),
 ('ходили', 0.9099265933036804),
 ('плохо', 0.9091436862945557),
 ('встретила', 0.9086582660675049),
 ('жалко', 0.9069492816925049),
 ('соскучилась', 0.906597375869751),
 ('кажется', 0.9034482836723328)]

In [21]:
def chuncks(gen, chunck_size=100):
    iterator = iter(gen)
    flag = True
    while flag:
        res = []
        for _ in range(chunck_size):
            try:
                res.append(next(iterator)) 
            except StopIteration:
                flag = False
                break
        yield res   


for chunc in chuncks(range(20), 3):
    print(chunc)

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
[9, 10, 11]
[12, 13, 14]
[15, 16, 17]
[18, 19]


In [40]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=10, max_iter=300)

def get_text_vector(text: str):
    return sum(np.array([w2v_model.wv[word] if (word in w2v_model.wv) else np.zeros(1000) for word in text.split() ]))

def vectors_generator(texts):
    for text in texts:
        yield get_text_vector(text)

j = 0
for batch in chuncks(zip(vectors_generator(df['clear']), df['target']), 3000):
#     print([(len(item), target) for item, target in batch])
    mlp.partial_fit(*list(zip(*batch)), classes=[0,1])
    j += 1
    if j % 10 == 0: 
        print(f"{j}/{len(df)//3000} done...\r", end='')
    
# mlp.partial_fit()
# df_vec = pd.DataFrame(df[['clear', 'target']], columns=['clear', 'target', *list(w2v_model.wv.vocab.keys())])

# df_vec.head()

In [45]:
test_text = """Магазин «Шар идеальная форма» предлагает одежду больших размеров дамам, имеющим завидные формы. «Шар идеальная форма» является уникальной торговой площадкой с широким ассортиментом от производителей из разных стран. В магазине собраны модели от известных дизайнеров Турции, России и Беларуси. Магазин «Шар идеальная форма я» позволяет любой моднице получить желаемый товар по доступным ценам. Спешите оформить заказ, и менеджеры мигом доставят его вам. Для постоянных клиентов действует система скидок."""
test_text

'Магазин «Шар идеальная форма» предлагает одежду больших размеров дамам, имеющим завидные формы. «Шар идеальная форма» является уникальной торговой площадкой с широким ассортиментом от производителей из разных стран. В магазине собраны модели от известных дизайнеров Турции, России и Беларуси. Магазин «Шар идеальная форма я» позволяет любой моднице получить желаемый товар по доступным ценам. Спешите оформить заказ, и менеджеры мигом доставят его вам. Для постоянных клиентов действует система скидок.'

In [46]:
#test_non_ad = """Будущий богослов родился в небольшом городке Аудеватер в южной части Испанских Нидерландов. Получив начальное образование на родине, продолжил изучать богословие в университетах Марбурга, Лейдена, Женевы и Базеля. В Женеве преподавателем Арминия стал известный протестантский богослов Теодор Беза, сподвижник Жана Кальвина. Полученные рекомендации позволили Арминию получить должность пастора в Амстердаме в 1588 году"""
test_non_ad = """И сын, так же как и муж, произвел в Анне чувство, похожее на разочарование. Она воображала его лучше, чем он был в действительности. Она была должна опуститься до действительности, чтобы наслаждаться им таким, каков он был. Но и такой, каков он был, он был прелестен с своими белокурыми кудрями, голубыми глазами и полными стройными ножками в туго натянутых чулках"""

test_non_ad

'И сын, так же как и муж, произвел в Анне чувство, похожее на разочарование. Она воображала его лучше, чем он был в действительности. Она была должна опуститься до действительности, чтобы наслаждаться им таким, каков он был. Но и такой, каков он был, он был прелестен с своими белокурыми кудрями, голубыми глазами и полными стройными ножками в туго натянутых чулках'

In [47]:
test_non_ad2 = """Полагая, что Арминий является сторонником доктрины о свободе воли и деятельном участии человека в деле своего спасения, Гомарус обвинял его в ереси пелагианства и полупелагианства. Их конфликт неоднократно становился предметом разбирательства гражданских властей республики, и своё самое известное произведение, «Объявление мнения», Арминий представил 20 октября 1608 года в своей речи перед Генеральными штатами. Богослов скончался в разгар споров относительно своего учения, и его последователи, получившие название ремонстрантов, организационно оформились после его смерти. """
test_non_ad2

'Полагая, что Арминий является сторонником доктрины о свободе воли и деятельном участии человека в деле своего спасения, Гомарус обвинял его в ереси пелагианства и полупелагианства. Их конфликт неоднократно становился предметом разбирательства гражданских властей республики, и своё самое известное произведение, «Объявление мнения», Арминий представил 20 октября 1608 года в своей речи перед Генеральными штатами. Богослов скончался в разгар споров относительно своего учения, и его последователи, получившие название ремонстрантов, организационно оформились после его смерти. '

In [48]:
mlp.predict([get_text_vector(text) for text in (test_text, test_non_ad, test_non_ad2)])

array([1, 0, 0])